# Zadanie klasyfikacji tekstu

W tym module rozpoczniemy od prostego zadania klasyfikacji tekstu opartego na zbiorze danych **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: będziemy klasyfikować nagłówki wiadomości do jednej z 4 kategorii: Świat, Sport, Biznes oraz Nauka/Technologia.

## Zbiór danych

Aby załadować zbiór danych, skorzystamy z API **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Możemy teraz uzyskać dostęp do części treningowej i testowej zbioru danych, używając odpowiednio `dataset['train']` i `dataset['test']:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Wydrukujmy pierwsze 10 nowych nagłówków z naszego zestawu danych:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Wektoryzacja tekstu

Teraz musimy przekształcić tekst w **liczby**, które mogą być reprezentowane jako tensory. Jeśli chcemy reprezentację na poziomie słów, musimy wykonać dwa kroki:

* Użyć **tokenizatora**, aby podzielić tekst na **tokeny**.
* Zbudować **słownik** tych tokenów.

### Ograniczanie rozmiaru słownika

W przykładzie z zestawem danych AG News rozmiar słownika jest dość duży, ponad 100 tysięcy słów. Ogólnie rzecz biorąc, nie potrzebujemy słów, które rzadko występują w tekście — tylko kilka zdań będzie je zawierać, a model nie nauczy się na ich podstawie. Dlatego warto ograniczyć rozmiar słownika do mniejszej liczby, przekazując odpowiedni argument do konstruktora wektoryzatora:

Oba te kroki można wykonać za pomocą warstwy **TextVectorization**. Zainicjujmy obiekt wektoryzatora, a następnie wywołajmy metodę `adapt`, aby przejść przez cały tekst i zbudować słownik:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Uwaga** używamy jedynie podzbioru całego zbioru danych, aby zbudować słownictwo. Robimy to, aby przyspieszyć czas wykonania i nie trzymać Cię w oczekiwaniu. Jednak podejmujemy ryzyko, że niektóre słowa z całego zbioru danych nie zostaną uwzględnione w słownictwie i będą pominięte podczas treningu. Zatem użycie pełnego rozmiaru słownictwa i przejście przez cały zbiór danych podczas `adapt` powinno zwiększyć ostateczną dokładność, ale nieznacznie.

Teraz możemy uzyskać dostęp do rzeczywistego słownictwa:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Korzystając z wektoryzatora, możemy łatwo zakodować dowolny tekst w zestaw liczb:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Reprezentacja tekstu za pomocą modelu Bag-of-words

Ponieważ słowa niosą znaczenie, czasami możemy zrozumieć sens fragmentu tekstu, patrząc jedynie na pojedyncze słowa, niezależnie od ich kolejności w zdaniu. Na przykład, przy klasyfikacji wiadomości, słowa takie jak *pogoda* i *śnieg* prawdopodobnie wskazują na *prognozę pogody*, podczas gdy słowa takie jak *akcje* i *dolar* będą związane z *wiadomościami finansowymi*.

Reprezentacja wektorowa **Bag-of-words** (BoW) jest najprostszą do zrozumienia tradycyjną metodą reprezentacji wektorowej. Każde słowo jest powiązane z indeksem wektora, a element wektora zawiera liczbę wystąpień danego słowa w określonym dokumencie.

![Obraz pokazujący, jak reprezentacja wektorowa bag-of-words jest przechowywana w pamięci.](../../../../../lessons/5-NLP/13-TextRep/images/bag-of-words-example.png) 

> **Note**: Możesz również myśleć o BoW jako o sumie wszystkich wektorów zakodowanych metodą one-hot dla poszczególnych słów w tekście.

Poniżej znajduje się przykład generowania reprezentacji bag-of-words przy użyciu biblioteki Scikit Learn w Pythonie:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Możemy również użyć wektoryzatora Keras, który zdefiniowaliśmy powyżej, konwertując każdy numer słowa na kodowanie one-hot i sumując wszystkie te wektory.


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Uwaga**: Możesz być zaskoczony, że wynik różni się od poprzedniego przykładu. Powodem jest to, że w przykładzie z Keras długość wektora odpowiada rozmiarowi słownika, który został zbudowany na podstawie całego zbioru danych AG News, podczas gdy w przykładzie z Scikit Learn słownik został zbudowany dynamicznie na podstawie próbki tekstu.


## Trenowanie klasyfikatora BoW

Teraz, gdy nauczyliśmy się, jak stworzyć reprezentację tekstu w formie bag-of-words, przejdźmy do trenowania klasyfikatora, który ją wykorzystuje. Najpierw musimy przekształcić nasz zbiór danych na reprezentację bag-of-words. Można to zrobić za pomocą funkcji `map` w następujący sposób:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Teraz zdefiniujmy prostą sieć neuronową klasyfikatora, która zawiera jedną warstwę liniową. Rozmiar wejścia to `vocab_size`, a rozmiar wyjścia odpowiada liczbie klas (4). Ponieważ rozwiązujemy zadanie klasyfikacji, końcową funkcją aktywacji jest **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Ponieważ mamy 4 klasy, dokładność powyżej 80% jest dobrym wynikiem.

## Trenowanie klasyfikatora jako jednej sieci

Ponieważ wektoryzator jest również warstwą Keras, możemy zdefiniować sieć, która go zawiera, i trenować ją od początku do końca. W ten sposób nie musimy wektoryzować zbioru danych za pomocą `map`, możemy po prostu przekazać oryginalny zbiór danych na wejście sieci.

> **Note**: Nadal musielibyśmy zastosować mapy do naszego zbioru danych, aby przekształcić pola z słowników (takich jak `title`, `description` i `label`) na krotki. Jednak podczas ładowania danych z dysku możemy od razu zbudować zbiór danych o wymaganej strukturze.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigramy, trigramy i n-gramy

Jednym z ograniczeń podejścia typu bag-of-words jest fakt, że niektóre słowa są częścią wyrażeń wielowyrazowych. Na przykład słowo „hot dog” ma zupełnie inne znaczenie niż słowa „hot” i „dog” w innych kontekstach. Jeśli zawsze reprezentujemy słowa „hot” i „dog” za pomocą tych samych wektorów, może to wprowadzać zamieszanie w naszym modelu.

Aby rozwiązać ten problem, często stosuje się **reprezentacje n-gramów** w metodach klasyfikacji dokumentów, gdzie częstotliwość każdego słowa, dwuwyrazu lub trójwyrazu jest przydatną cechą do trenowania klasyfikatorów. W reprezentacjach bigramowych, na przykład, dodajemy wszystkie pary słów do słownika, oprócz oryginalnych słów.

Poniżej znajduje się przykład, jak wygenerować reprezentację bag-of-words dla bigramów za pomocą Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Główną wadą podejścia n-gramowego jest to, że rozmiar słownika zaczyna rosnąć niezwykle szybko. W praktyce musimy połączyć reprezentację n-gramową z techniką redukcji wymiarów, taką jak *embeddings*, którą omówimy w następnej jednostce.

Aby użyć reprezentacji n-gramowej w naszym zbiorze danych **AG News**, musimy przekazać parametr `ngrams` do konstruktora `TextVectorization`. Rozmiar słownika bigramów jest **znacznie większy**, w naszym przypadku wynosi ponad 1,3 miliona tokenów! Dlatego warto ograniczyć liczbę tokenów bigramowych do jakiejś rozsądnej wartości.

Moglibyśmy użyć tego samego kodu co powyżej, aby wytrenować klasyfikator, jednak byłoby to bardzo nieefektywne pod względem pamięci. W następnej jednostce wytrenujemy klasyfikator bigramowy, korzystając z embeddings. W międzyczasie możesz eksperymentować z treningiem klasyfikatora bigramowego w tym notebooku i sprawdzić, czy uda Ci się osiągnąć wyższą dokładność.


## Automatyczne obliczanie wektorów BoW

W powyższym przykładzie obliczyliśmy wektory BoW ręcznie, sumując jednorazowe kodowania poszczególnych słów. Jednak najnowsza wersja TensorFlow pozwala na automatyczne obliczanie wektorów BoW poprzez przekazanie parametru `output_mode='count` do konstruktora wektoryzatora. Dzięki temu definiowanie i trenowanie naszego modelu staje się znacznie łatwiejsze:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Częstotliwość terminu - odwrotna częstotliwość dokumentu (TF-IDF)

W reprezentacji BoW wystąpienia słów są ważone w ten sam sposób, niezależnie od samego słowa. Jednak oczywiste jest, że częste słowa, takie jak *a* i *w*, są znacznie mniej istotne dla klasyfikacji niż terminy specjalistyczne. W większości zadań NLP niektóre słowa są bardziej istotne niż inne.

**TF-IDF** oznacza **częstotliwość terminu - odwrotna częstotliwość dokumentu**. Jest to wariacja metody bag-of-words, w której zamiast binarnej wartości 0/1 wskazującej na obecność słowa w dokumencie, używana jest wartość zmiennoprzecinkowa, która jest związana z częstotliwością występowania słowa w korpusie.

Bardziej formalnie, waga $w_{ij}$ słowa $i$ w dokumencie $j$ jest zdefiniowana jako:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
gdzie
* $tf_{ij}$ to liczba wystąpień $i$ w $j$, czyli wartość BoW, którą widzieliśmy wcześniej
* $N$ to liczba dokumentów w kolekcji
* $df_i$ to liczba dokumentów zawierających słowo $i$ w całej kolekcji

Wartość TF-IDF $w_{ij}$ rośnie proporcjonalnie do liczby wystąpień słowa w dokumencie i jest korygowana przez liczbę dokumentów w korpusie, które zawierają to słowo, co pomaga uwzględnić fakt, że niektóre słowa pojawiają się częściej niż inne. Na przykład, jeśli słowo pojawia się w *każdym* dokumencie w kolekcji, $df_i=N$, a $w_{ij}=0$, i takie terminy są całkowicie pomijane.

Możesz łatwo stworzyć wektoryzację TF-IDF tekstu za pomocą Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

W Keras warstwa `TextVectorization` może automatycznie obliczać częstotliwości TF-IDF, przekazując parametr `output_mode='tf-idf'`. Powtórzmy kod, którego użyliśmy powyżej, aby sprawdzić, czy użycie TF-IDF zwiększa dokładność:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Podsumowanie

Chociaż reprezentacje TF-IDF przypisują wagi częstotliwościowe różnym słowom, nie są w stanie oddać znaczenia ani kolejności. Jak powiedział słynny językoznawca J. R. Firth w 1935 roku: "Pełne znaczenie słowa zawsze zależy od kontekstu, a żadna analiza znaczenia oderwana od kontekstu nie może być traktowana poważnie." W dalszej części kursu nauczymy się, jak uchwycić informacje kontekstowe z tekstu za pomocą modelowania języka.



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż staramy się zapewnić dokładność, prosimy mieć na uwadze, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji krytycznych zaleca się skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
